In [1]:
import numpy as np
import cv2
import pandas as pd
import os
import pickle
from google.colab import drive
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

In [2]:
# Connect Google Drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/test_opencv/'
# Hiển thị tất cả thư mục hoặc file
os.listdir(path)

Mounted at /content/drive


['face1.jpg',
 'face3.jpg',
 'face2.jpg',
 'models',
 'FaceImage',
 'data_face_features.pickle',
 'project']

In [3]:
data = pickle.load(open("/content/drive/MyDrive/test_opencv/data_face_features.pickle", mode="rb"))

In [4]:
x = np.array(data['data'])
y = np.array(data['label'])

print(x)
print(y)

[[[ 0.02926855  0.21446565 -0.03790818 ... -0.06640885  0.01369472
    0.06734287]]

 [[-0.01456819  0.00786272  0.08517674 ...  0.11353574  0.04404245
    0.12897009]]

 [[ 0.05722998 -0.01173182 -0.03553959 ...  0.15690751 -0.08634205
    0.00904024]]

 ...

 [[ 0.04699203  0.23070885  0.129168   ...  0.01667671  0.22774613
    0.05523583]]

 [[ 0.0438237   0.16595688  0.08894051 ... -0.02917577  0.13665205
    0.03569263]]

 [[ 0.02900951  0.17730097  0.11366539 ... -0.04183742  0.18185957
    0.05536526]]]
['151901205_BuiKhanhBinh' '151901205_BuiKhanhBinh'
 '151901205_BuiKhanhBinh' ... '1721031731_NguyenDongHung'
 '1721031731_NguyenDongHung' '1721031731_NguyenDongHung']


In [5]:
x.shape

(4694, 1, 128)

In [6]:
y.shape
x = x.reshape(-1, 128)
x.shape
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=1)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((3755, 128), (939, 128), (3755,), (939,))

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [8]:
def get_report(model, x_train, y_train, x_test, y_test):
  y_pred_train = model.predict(x_train)
  y_pred_test = model.predict(x_test)

  # accuracy score
  acc_train = accuracy_score(y_train, y_pred_train)
  acc_test = accuracy_score(y_test, y_pred_test)

  # f1 score
  f1_score_train = f1_score(y_train, y_pred_train, average="macro")
  f1_score_test = f1_score(y_test, y_pred_test, average="macro")

  # print
  print('Accurency Train = %0.2f'%acc_train)
  print('Accurency Test = %0.2f'%acc_test)
  print('F1 Score Train = %0.2f'%f1_score_train)
  print('F1 Score Test = %0.2f'%f1_score_test)

In [9]:
# Logistic regression
model_logistic = LogisticRegression()
model_logistic.fit(x_train, y_train) # training

LogisticRegression()

In [10]:
get_report(model_logistic, x_train, y_train, x_test, y_test)

Accurency Train = 0.72
Accurency Test = 0.69
F1 Score Train = 0.52
F1 Score Test = 0.49


In [11]:
model_svc = SVC(probability=True)
model_svc.fit(x_train, y_train)
get_report(model_svc, x_train, y_train, x_test, y_test)
model_rf = RandomForestClassifier(n_estimators=20,)
model_rf.fit(x_train, y_train)
get_report(model_rf, x_train, y_train, x_test, y_test)

Accurency Train = 0.91
Accurency Test = 0.84
F1 Score Train = 0.83
F1 Score Test = 0.73
Accurency Train = 1.00
Accurency Test = 0.77
F1 Score Train = 1.00
F1 Score Test = 0.67


In [12]:
model_voting = VotingClassifier(estimators=[
    ('logistic', LogisticRegression()),
    ('svm', SVC(probability=True)),
    ('rf', RandomForestClassifier(n_estimators=20,)),
], voting='soft', weights=[2,3,1])

In [13]:
model_voting.fit(x_train, y_train)
get_report(model_voting, x_train, y_train, x_test, y_test)

Accurency Train = 0.98
Accurency Test = 0.84
F1 Score Train = 0.97
F1 Score Test = 0.74


In [14]:
from sklearn.model_selection import GridSearchCV

model_grid = GridSearchCV(model_voting, param_grid={
    'svm__C': [3,5,7,10],
    'svm__gamma': [0.1, 0.3, 0.5],
    'rf__n_estimators': [5, 10,20],
    'rf__max_depth': [3, 5, 7],
    'voting': ['soft', 'hard']
}, scoring='accuracy', cv=3, n_jobs=1, verbose=2)

In [15]:
model_grid.fit(x_train, y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=  11.1s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   6.8s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   7.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   6.8s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   5.9s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   8.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   5.8s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   7.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   5.6s
[

GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier(n_estimators=20))],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [16]:
model_best_estimator = model_grid.best_estimator_

In [17]:
model_grid.best_score_

0.8447406079396332

In [18]:
pickle.dump(model_best_estimator, open("/content/drive/MyDrive/test_opencv/ml_face_person_identity.pkl", mode='wb'))